In [ ]:
Подключение google disk для использования dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Установка библиотеки

In [2]:
!pip install petroscope

Импорт неободимых библиотек

In [3]:
from pathlib import Path
from petroscope.segmentation.classes import ClassSet, LumenStoneClasses
from petroscope.segmentation.utils import load_image, load_mask
from petroscope.segmentation import GeoSegmModel
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from skimage.color import rgb2lab
import os

Путь к dataset

In [4]:
ds_path = Path('/content/drive/MyDrive/PhotoSets/')

Инциализация набора классов для сегментации

In [5]:
classset = LumenStoneClasses.S1v1()

for cl in classset.classes:
    print(cl)

[0, bg (background), color: #000000]
[1, ccp/kub (chalcopyrite/cubanite), color: #ffa500]
[2, gl (galena), color: #9acd32]
[4, brt (bornite), color: #00bfff]
[6, py/mrc (pyrite/marcasite), color: #2f4f4f]
[8, sph (sphalerite), color: #ee82ee]
[11, tnt/ttr (tenantite/tetrahedrite), color: #483d8b]


Формирование путей к изображениям и маскам

In [6]:
train_img_mask_p = [
    (img_p, ds_path / "masks" / "train" / f"{img_p.stem}.png")
    for img_p in sorted((ds_path / "imgs" / "train").iterdir())
]

test_img_mask_p = [
    (img_p, ds_path / "masks" / "test" / f"{img_p.stem}.png")
    for img_p in sorted((ds_path / "imgs" / "test").iterdir())
]

Загрузка и конвертация обучающих изображений в пространство LAB

In [7]:
for img_p, _ in train_img_mask_p:
    img = load_image(img_p, normalize=True)
    img_lab = rgb2lab(img)
    print(f"Image {img_p.name}: {img_lab.shape}, {img_lab.dtype}")

Image train_01.jpg: (2547, 3396, 3), float32
Image train_02.jpg: (2547, 3396, 3), float32
Image train_03.jpg: (2547, 3396, 3), float32
Image train_04.jpg: (2547, 3396, 3), float32
Image train_06.jpg: (2547, 3396, 3), float32
Image train_07.jpg: (2547, 3396, 3), float32
Image train_08.jpg: (2547, 3396, 3), float32
Image train_09.jpg: (2547, 3396, 3), float32
Image train_10.jpg: (2547, 3396, 3), float32
Image train_11.jpg: (2547, 3396, 3), float32
Image train_12.jpg: (2547, 3396, 3), float32
Image train_13.jpg: (2547, 3396, 3), float32
Image train_14.jpg: (2547, 3396, 3), float32
Image train_15.jpg: (2547, 3396, 3), float32
Image train_16.jpg: (2547, 3396, 3), float32
Image train_17.jpg: (2547, 3396, 3), float32
Image train_18.jpg: (2547, 3396, 3), float32
Image train_19.jpg: (2547, 3396, 3), float32
Image train_20.jpg: (2547, 3396, 3), float32
Image train_21.jpg: (2547, 3396, 3), float32
Image train_22.jpg: (2547, 3396, 3), float32
Image train_23.jpg: (2547, 3396, 3), float32
Image trai

Загрузка масок без one-hot кодирования

In [8]:
for _, mask_p in train_img_mask_p:
    mask = load_mask(mask_p, classes=classset, one_hot=False)
    print(f"Mask {mask_p.name}: {mask.shape}, {mask.dtype}")

Mask train_01.png: (2547, 3396), uint8
Mask train_02.png: (2547, 3396), uint8
Mask train_03.png: (2547, 3396), uint8
Mask train_04.png: (2547, 3396), uint8
Mask train_06.png: (2547, 3396), uint8
Mask train_07.png: (2547, 3396), uint8
Mask train_08.png: (2547, 3396), uint8
Mask train_09.png: (2547, 3396), uint8
Mask train_10.png: (2547, 3396), uint8
Mask train_11.png: (2547, 3396), uint8
Mask train_12.png: (2547, 3396), uint8
Mask train_13.png: (2547, 3396), uint8
Mask train_14.png: (2547, 3396), uint8
Mask train_15.png: (2547, 3396), uint8
Mask train_16.png: (2547, 3396), uint8
Mask train_17.png: (2547, 3396), uint8
Mask train_18.png: (2547, 3396), uint8
Mask train_19.png: (2547, 3396), uint8
Mask train_20.png: (2547, 3396), uint8
Mask train_21.png: (2547, 3396), uint8
Mask train_22.png: (2547, 3396), uint8
Mask train_23.png: (2547, 3396), uint8
Mask train_24.png: (2547, 3396), uint8
Mask train_25.png: (2547, 3396), uint8
Mask train_26.png: (2547, 3396), uint8
Mask train_27.png: (2547,

Загрузка масок с one-hot кодированием

In [9]:
for _, mask_p in train_img_mask_p:
    mask_one_hot = load_mask(mask_p, classes=classset, one_hot=True)
    print(f"Mask one-hot {mask_p.name}: {mask_one_hot.shape}, {mask_one_hot.dtype}")

Mask one-hot train_01.png: (2547, 3396, 7), float32
Mask one-hot train_02.png: (2547, 3396, 7), float32
Mask one-hot train_03.png: (2547, 3396, 7), float32
Mask one-hot train_04.png: (2547, 3396, 7), float32
Mask one-hot train_06.png: (2547, 3396, 7), float32
Mask one-hot train_07.png: (2547, 3396, 7), float32
Mask one-hot train_08.png: (2547, 3396, 7), float32
Mask one-hot train_09.png: (2547, 3396, 7), float32
Mask one-hot train_10.png: (2547, 3396, 7), float32
Mask one-hot train_11.png: (2547, 3396, 7), float32
Mask one-hot train_12.png: (2547, 3396, 7), float32
Mask one-hot train_13.png: (2547, 3396, 7), float32
Mask one-hot train_14.png: (2547, 3396, 7), float32
Mask one-hot train_15.png: (2547, 3396, 7), float32
Mask one-hot train_16.png: (2547, 3396, 7), float32
Mask one-hot train_17.png: (2547, 3396, 7), float32
Mask one-hot train_18.png: (2547, 3396, 7), float32
Mask one-hot train_19.png: (2547, 3396, 7), float32
Mask one-hot train_20.png: (2547, 3396, 7), float32
Mask one-hot

Загрузка цветных масок

In [10]:
for img_p, _ in train_img_mask_p:
    mask_colored_path = ds_path / "masks_colored_png" / "train" / f"{img_p.stem}.png"
    mask_colored = load_image(mask_colored_path, normalize=False)
    print(f"Colored mask {mask_colored_path.name}: {mask_colored.shape}, {mask_colored.dtype}")

Colored mask train_01.png: (2547, 3396, 3), uint8
Colored mask train_02.png: (2547, 3396, 3), uint8
Colored mask train_03.png: (2547, 3396, 3), uint8
Colored mask train_04.png: (2547, 3396, 3), uint8
Colored mask train_06.png: (2547, 3396, 3), uint8
Colored mask train_07.png: (2547, 3396, 3), uint8
Colored mask train_08.png: (2547, 3396, 3), uint8
Colored mask train_09.png: (2547, 3396, 3), uint8
Colored mask train_10.png: (2547, 3396, 3), uint8
Colored mask train_11.png: (2547, 3396, 3), uint8
Colored mask train_12.png: (2547, 3396, 3), uint8
Colored mask train_13.png: (2547, 3396, 3), uint8
Colored mask train_14.png: (2547, 3396, 3), uint8
Colored mask train_15.png: (2547, 3396, 3), uint8
Colored mask train_16.png: (2547, 3396, 3), uint8
Colored mask train_17.png: (2547, 3396, 3), uint8
Colored mask train_18.png: (2547, 3396, 3), uint8
Colored mask train_19.png: (2547, 3396, 3), uint8
Colored mask train_20.png: (2547, 3396, 3), uint8
Colored mask train_21.png: (2547, 3396, 3), uint8


Функция для загрузки и предобработки изображений и масок

In [11]:
def load_and_preprocess(img_path, mask_path, classes, img_size=(256, 256)):
    img = load_image(img_path, normalize=True)
    img = tf.image.resize(img, img_size)

    mask = load_mask(mask_path, classes=classes, one_hot=False)
    mask = tf.image.resize(mask[..., np.newaxis], img_size, method='nearest')
    mask = to_categorical(mask, num_classes=len(classes))

    return img, mask

Унаследованный класс от GeoSegmModel, метод сегментации - нейронная сеть U-Net

In [12]:
class UNetSegmModel(GeoSegmModel):
    def __init__(self, classes: ClassSet, input_size=(256, 256, 3), save_path="unet_model"):
        super().__init__()
        self.classes = classes
        self.input_size = input_size
        self.save_path = Path(save_path)
        self.model = self._build_model()

    def _build_model(self):
        inputs = Input(self.input_size)

        conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
        conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
        conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
        conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
        conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
        conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)

        up5 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv3], axis=-1)
        conv5 = Conv2D(256, 3, activation='relu', padding='same')(up5)
        conv5 = Conv2D(256, 3, activation='relu', padding='same')(conv5)
        up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv2], axis=-1)
        conv6 = Conv2D(128, 3, activation='relu', padding='same')(up6)
        conv6 = Conv2D(128, 3, activation='relu', padding='same')(conv6)
        up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv1], axis=-1)
        conv7 = Conv2D(64, 3, activation='relu', padding='same')(up7)
        conv7 = Conv2D(64, 3, activation='relu', padding='same')(conv7)

        outputs = Conv2D(len(self.classes), 1, activation='softmax')(conv7)

        model = Model(inputs=[inputs], outputs=[outputs])
        model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    def train(self, img_mask_paths, epochs=20, batch_size=4, validation_split=0.1, **kwargs):
        train_data = [load_and_preprocess(img_p, mask_p, self.classes) for img_p, mask_p in tqdm(img_mask_paths, desc="Loading data")]
        train_images, train_masks = zip(*train_data)
        train_images = np.array(train_images)
        train_masks = np.array(train_masks)

        self.model.fit([train_images], [train_masks], batch_size=batch_size, epochs=epochs, validation_split=validation_split)

        self.save()

    def predict_image(self, image: np.ndarray) -> np.ndarray:
        img_resized = tf.image.resize(image, self.input_size[:2])
        pred = self.model.predict(np.expand_dims(img_resized, axis=0))
        pred = np.argmax(pred[0], axis=-1)
        return pred.astype(np.uint8)

    def save(self):
        self.save_path.mkdir(parents=True, exist_ok=True)
        self.model.save(self.save_path / "unet_model.keras")

    def load(self, saved_path: Path, **kwargs):
        if not saved_path.exists():
            raise FileNotFoundError(f"Model file not found at {saved_path}")
        self.model = load_model(saved_path / "unet_model.keras")

Обучение модели

In [13]:
model = UNetSegmModel(classes=classset, input_size=(256, 256, 3), save_path="unet_model")

model.train(train_img_mask_p, epochs=200, batch_size=8, validation_split=0.1)

Loading data: 100%|██████████| 58/58 [00:32<00:00,  1.77it/s]


Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 81s 6s/step - accuracy: 0.2878 - loss: 3.1463 - val_accuracy: 0.3438 - val_loss: 1.9293
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 335ms/step - accuracy: 0.3057 - loss: 1.7832 - val_accuracy: 0.3515 - val_loss: 2.0551
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 345ms/step - accuracy: 0.4016 - loss: 1.4435 - val_accuracy: 0.3582 - val_loss: 1.7751
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 337ms/step - accuracy: 0.5407 - loss: 1.2612 - val_accuracy: 0.4854 - val_loss: 1.7893
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 336ms/step - accuracy: 0.6613 - loss: 1.1003 - val_accuracy: 0.4419 - val_loss: 2.0052
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 336ms/step - accuracy: 0.6560 - loss: 1.0465 - val_accuracy: 0.4682 - val_loss: 1.8772
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 335ms/step - accuracy: 0.7034 - loss: 0.9920 - val_accuracy: 0.4710 - val_loss: 1.8852
Epoch 8/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 350ms/step - accuracy: 0.7238 - loss: 0.9609 - val_accuracy: 0.4408 - val

Тестирование

In [14]:
from petroscope.segmentation.eval import SegmDetailedTester
from tensorflow.keras.utils import to_categorical

tester = SegmDetailedTester(
    Path("output"),
    classes=classset,
    void_pad=0,
    void_border_width=4,
    vis_plots=False,
    vis_segmentation=True,
)

for img_p, mask_p in test_img_mask_p:
    img = load_image(img_p, normalize=True)

    pred = model.predict_image(img)
    print(f"Pred shape: {pred.shape}")

    mask = load_mask(mask_p, classes=classset, one_hot=False)
    mask_resized = tf.image.resize(mask[..., np.newaxis], (256, 256), method='nearest')
    mask_resized = mask_resized[..., 0].numpy().astype(np.uint8)
    print(f"Mask resized shape: {mask_resized.shape}")

    pred_one_hot = to_categorical(pred, num_classes=len(classset.classes))
    mask_one_hot = to_categorical(mask_resized, num_classes=len(classset.classes))
    print(f"Pred one-hot shape: {pred_one_hot.shape}")
    print(f"Mask one-hot shape: {mask_one_hot.shape}")

    metrics = tester.eval.evaluate(pred_one_hot, gt=mask_one_hot)
    metrics_void = tester.eval_void.evaluate(pred_one_hot, gt=mask_one_hot)

    print(f"Metrics for {img_p.name}:\n{metrics}")
    print(f"Metrics with void borders for {img_p.name}:\n{metrics_void}")
    print("-" * 50)

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(1, 256, 256, 3))
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Pred shape: (256, 256)
Mask resized shape: (256, 256)
Pred one-hot shape: (256, 256, 7)
Mask one-hot shape: (256, 256, 7)
Metrics for test_01.jpg:
	 iou [soft]:
		 bg: 0.8814 [0.8814]
		 brt: 0.0010 [0.0010]
		 ccp/kub: 0.7368 [0.7368]
		 gl: 0.1642 [0.1642]
		 py/mrc: 0.9581 [0.9581]
		 sph: 0.7849 [0.7849]
		 tnt/ttr: 0.0074 [0.0074]
	 mean iou [soft]: 0.5048 [0.5048]
	 acc: 0.9280

Metrics with void borders for test_01.jpg:
	 iou [soft]:
		 bg: 0.8814 [0.8814]
		 brt: 0.0010 [0.0010]
		 ccp/kub: 0.7368 [0.7368]
		 gl: 0.1642 [0.1642]
		 py/mrc: 0.9581 [0.9581]
		 sph: 0.7849 [0.7849]
		 tnt/ttr: 0.0074 [0.0074]
	 mean iou [soft]: 0.5048 [0.5048]
	 acc: 0.9280

--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Pred shape: (256, 256)
Mask resized shape: (256, 256)
Pred one-hot shape: (256, 256, 7)
Mask one-hot shape: (256, 256, 7)
Metrics for test_02.jpg:
	 iou [soft]:
		 bg: 0.7972 [0.7972]
		 brt: 1.0000 [1.0